# Part 1

In [5]:
import numpy as np

In [6]:
class moonsOfJupiter():
    def __init__(self, input_line):
        self.position = self.parse_input(input_line)
        self.velocity = [0, 0, 0]
        
        self.dv = [0, 0, 0]
        
        self.x_period, self.y_period, self.z_period = 0, 0, 0
        self.vx_period, self.vy_period, self.vz_period = 0, 0, 0
        
        self.xhistory, self.yhistory, self.zhistory = set(), set(), set()
        self.vxhistory, self.vyhistory, self.vzhistory = set(), set(), set()

    def __repr__(self):
        return (self.position[0], self.position[1], self.position[2], 
                self.velocity[0], self.velocity[1], self.velocity[2])
    
    def parse_input(self, input_line):
        input_line = input_line[1:-2].split(', ')
        
        x = int(input_line[0][2:])
        y = int(input_line[1][2:])
        z = int(input_line[2][2:])
        
        return [x, y, z]
    
    def calculate_v_update(self, moon1, moon2, moon3):
        for i in range(3):
            self.dv[i] = self.calculate_increment(self.position[i], moon1.position[i]) + \
                         self.calculate_increment(self.position[i], moon2.position[i]) + \
                         self.calculate_increment(self.position[i], moon3.position[i])
    
    def calculate_increment(self, one, two):
        if (one<two):
            return 1
        elif(one>two):
            return -1
        else:
            return 0
        
    def update_pv(self):
        for i in range(3):
            self.velocity[i] += self.dv[i]
        for i in range(3):
            self.position[i] += self.velocity[i]
            
        self.dv = [0, 0, 0]
        
    def calculate_pe(self):
        pe = 0
        for i in range(3):
            pe += abs(self.position[i]) 
        return pe
    
    def calculate_ke(self):
        ke = 0
        for i in range(3):
            ke += abs(self.velocity[i]) 
        return ke
    
    def calculate_te(self):
        '''calculate total energy'''
        total_energy = self.calculate_pe() * self.calculate_ke()
        return total_energy
    
    def print_pv(self):
        print([self.position[0], self.position[1], self.position[2], self.velocity[0], self.velocity[1], self.velocity[2]])
        
    def check_histories(self, i):
        
        if not self.x_period:
            if self.position[0] not in self.xhistory:
                self.xhistory.add(self.position[0])
            else:
                self.x_period = i
                
        if not self.y_period:
            if self.position[1] not in self.yhistory:
                self.yhistory.add(self.position[1])
            else:
                self.y_period = i

        if not self.z_period:
            if self.position[2] not in self.zhistory:
                self.zhistory.add(self.position[2])
            else:
                self.z_period = i

        if not self.vx_period:
            if self.velocity[0] not in self.vxhistory:
                self.vxhistory.add(self.velocity[0])
            else:
                self.vx_period = i
                
        if not self.vy_period:
            if self.velocity[1] not in self.vyhistory:
                self.vyhistory.add(self.velocity[1])
            else:
                self.vy_period = i

        if not self.vz_period:
            if self.velocity[2] not in self.vzhistory:
                self.vzhistory.add(self.velocity[2])
            else:
                self.vz_period = i

In [7]:
input_file = 'day12_input.txt'
# input_file = 'day12_test_input.txt'
with open(input_file, 'r') as f:
    lines = f.readlines()
    
# Setting initial positions and velocities for the moons
io = moonsOfJupiter(lines[0])
europa = moonsOfJupiter(lines[1])
ganymede = moonsOfJupiter(lines[2])
callisto = moonsOfJupiter(lines[3])

In [8]:
# Let's simulate the trajectories!
for i in range(1000):
    io.calculate_v_update(europa, ganymede, callisto)
    europa.calculate_v_update(io, ganymede, callisto)
    ganymede.calculate_v_update(io, europa, callisto)
    callisto.calculate_v_update(io, europa, ganymede)
    
    io.update_pv()
    europa.update_pv()
    ganymede.update_pv()
    callisto.update_pv() 

In [9]:
print(io.calculate_te() + ganymede.calculate_te() + europa.calculate_te() + callisto.calculate_te())

11384


# Part 2

In [141]:
input_file = 'day12_input.txt'
# input_file = 'day12_test_input.txt'
with open(input_file, 'r') as f:
    lines = f.readlines()
    
# Setting initial positions and velocities for the moons
io = moonsOfJupiter(lines[0])
europa = moonsOfJupiter(lines[1])
ganymede = moonsOfJupiter(lines[2])
callisto = moonsOfJupiter(lines[3])

io_initial = io.__repr__()
europa_initial = europa.__repr__()
ganymede_initial = ganymede.__repr__()
callisto_initial = callisto.__repr__()

In [142]:
io_periods = [[0], [0], [0]]
europa_periods = [[0], [0], [0]]
ganymede_periods = [[0], [0], [0]]
callisto_periods = [[0], [0], [0]]

In [143]:
def fillPeriods(periods, inputValue, refValue, i):
    
    for j in range(3):
        if((inputValue[j] == refValue[j]) and (inputValue[j+3] == refValue[j+3])):
            periods[j].append(i)
    
    return periods

In [144]:
def calcPeriodsHelper(axisPeriods):
    i, period = 1, 0
    
    while(axisPeriods[i] != axisPeriods[0]):
        period += axisPeriods[i]
        i += 1
    
    period += axisPeriods[0] if (i == 1) else 2*axisPeriods[0]
    
    return period

In [145]:
def calcPeriods(periods):
    ''' given the periods in x, y, and z, return the period of the moon'''
    
    periods_x = (np.array(periods[0][1:]) - np.array(periods[0][0:-1]))
    period_x = calcPeriodsHelper(periods_x)
        
    periods_y = (np.array(periods[1][1:]) - np.array(periods[1][0:-1]))
    period_y = calcPeriodsHelper(periods_y)
    
    periods_z = (np.array(periods[2][1:]) - np.array(periods[2][0:-1]))
    period_z = calcPeriodsHelper(periods_z)
    
    return np.lcm.reduce([period_x, period_y, period_z])

In [146]:
# Let's simulate the trajectories!
# Add while loop here that checks lengths of each periods, making sure we have at least 50 elements each
for i in range(1, 500000):
    io.calculate_v_update(europa, ganymede, callisto)
    europa.calculate_v_update(io, ganymede, callisto)
    ganymede.calculate_v_update(io, europa, callisto)
    callisto.calculate_v_update(io, europa, ganymede)
    
    io.update_pv()
    europa.update_pv()
    ganymede.update_pv()
    callisto.update_pv()
    
    io_periods = fillPeriods(io_periods, io.__repr__(), io_initial, i)
    europa_periods =fillPeriods(europa_periods, europa.__repr__(), europa_initial, i)
    ganymede_periods = fillPeriods(ganymede_periods, ganymede.__repr__(), ganymede_initial, i)
    callisto_periods = fillPeriods(callisto_periods, callisto.__repr__(), callisto_initial, i)

    

In [147]:
io_period = calcPeriods(io_periods)
europa_period = calcPeriods(europa_periods)
callisto_period = calcPeriods(callisto_periods)
ganymede_period = calcPeriods(ganymede_periods)

In [148]:
np.lcm.reduce([io_period, europa_period, callisto_period, ganymede_period])

452582583272768